In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import cv2
import json
import numpy as np
import xml.etree.ElementTree as et
import pandas as pd

from src.data.image_augmentation import (
    create_annotation_file,
    get_bbox,
    plot_img_with_bbox,
    write_bbox_to_csv,
    enlarge_dataset,
    remove_processed_imgs,
)

In [2]:
# Create csv annotation file at data/labeling
create_annotation_file()

In [3]:
# Optional: remove processed images from processed folder
remove_processed_imgs()

In [5]:
# Enlarge dataset by factor
enlarge_dataset(1)